# Нормализуем суммы в транзакциях   

Цель: предсказать для каждого пользователя взятие/ невзятие каждого из четырех продуктов **в течение месяца после отчетной даты**, исторические данные по ним находятся в targets

## Данные



### Transactions
|title|description|
|---|---|
|client_id|id клиента|
|amount|Сумма транзакции|
|event_time|Дата транзакции|
|event_type|Тип транзакции|
|event_subtype|Уточнение типа транзакции|
|currency|Валюта|
|src_type11|Признак 1 для отправителя|
|src_type12|Уточнение признака 1 для отправителя|
|dst_type11|Признак 1 для контрагента|
|dst_type12|Уточнение для признака 1 для контрагента|  
|src_type21|Признак 2 для отправителя|  
|src_type22|Уточнение признака 2 для отправителя|
|src_type31|Признак 3 для отправителя|
|src_type32|Уточнение признака 3 для отправителя|

### Внешние источники данных:  
- Открытые данные ЦБ РФ по инфляции  
График инфляции и ключ.ставки за каждый месяц доступны по ссылке: https://www.cbr.ru/hd_base/infl/  
Данные представлены за каждый месяц.    
На основе данных ЦБ по инфляции для каждого периода рассчитана в виде кумулятивного процента (накопительный процент с учетом прошлых месяцев).  
Далее составлен словарь для каждого месяца с указанием на сколько надо умножить сумму транзакции, чтобы привести все к текущим значениям. И данная операция была проведена над всеми данными.   

- Открытые данные ЦБ РФ по курсам валют.   
Курсы валют за каждый день доступны по ссылке: https://www.cbr.ru/currency_base/dynamics/   

Валюты представлены в зашифрованном виде:  
11.0    167351850   
1.0         47897  
14.0        39889  
7.0         21909  
17.0         2115   
9.0          2074  
….  
Можно привести самые первые транзакции к рублям, если сделать такое предположение, что чаще всего используются рубли (11), далее  это доллары (1) и евро (14), далее другая валюта. Другими популярными валютами являются, например, белорусские рубли и юани.   Кроме долларов и евро, курс у остальных валют колеблется в районе 20 руб.  
   
Т.к. данных довольно много, то было применено простое приведение сумм к рублям:  
- рубли (код 11), оставлен без изменения  
- валюты с кодами 1 и 14 были умножены на среднее значение доллара и евро за указанный период 2022 год это ~70 рублей  
- Остальная валюта умножена на значение 20  


In [97]:
import pandas as pd
from pandas.api.types import is_float_dtype, is_integer_dtype
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

import glob
import pyarrow.parquet as pq
from tqdm import trange, tqdm

In [3]:
from typing import List, Optional

In [4]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# найтройки
# Убираем ограничение отображемых колонок
pd.set_option("display.max_columns", None)
# Устанавливаем тему по умолчанию
sb_dark = sns.dark_palette('skyblue', 8, reverse=True) # teal
sns.set(palette=sb_dark)

In [6]:
# Включаем tqdm для pandas, чтобы можно было запускать progress_apply() вместо простого apply()
tqdm.pandas() 
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [7]:
eps = 1e-6

In [10]:
PATH = ''
PATH_DATASET = PATH + 'datasets/sber_source/'
PATH_DATASET_OUTPUT = PATH + 'datasets/'
PATH_DATASET_TRX_TRAIN = PATH_DATASET + 'trx_train.parquet/'
PATH_DATASET_TRX_TEST = PATH_DATASET + 'trx_test.parquet/'

PATH_DATASET_TARGET_TRAIN = PATH_DATASET + 'train_target.parquet/'
PATH_DATASET_TARGET_TEST = PATH_DATASET + 'test_target_b.parquet/'

# Определяем пути к данным транзакциям
train_trx_files = glob.glob(PATH_DATASET_TRX_TRAIN + '/*.parquet')
test_trx_files = glob.glob(PATH_DATASET_TRX_TEST + '/*.parquet')

# таргеты
train_target_files = glob.glob(PATH_DATASET_TARGET_TRAIN + '/*.parquet')
test_target_files = glob.glob(PATH_DATASET_TARGET_TEST + '/*.parquet')

len(train_trx_files), len(test_trx_files)

(14, 3)

In [11]:
# Загрузка списка файлов (типа паркет) в один датафрейм
def load_df_by_files(files:list[str]) -> pd.DataFrame:
    union_df = pd.DataFrame()
    for file in tqdm(files):
        current_df = pq.read_table(file).to_pandas()    
        union_df = pd.concat([union_df, current_df])
    return union_df

In [91]:
# %%time
# # Загружаем все таргеты
# all_target_df = load_df_by_files(train_target_files + test_target_files)
# all_target_df.shape

In [13]:
%%time
compress_trx_df = pq.read_table(PATH_DATASET_OUTPUT + 'compress_trx_df_06_06_2024.parquet').to_pandas()
compress_trx_df.shape

CPU times: total: 2min 20s
Wall time: 2min 58s


(215076361, 14)

## Нормализация сумм транзакций

1. Учитываем инфляцию  
2. Учитываем курс валют  
Валюты представлены в зашифрованном виде  
11.0    167351850  
1.0         47897  
14.0        39889  
7.0         21909    
17.0         2115  
9.0          2074  
….
Можно привести самые первые транзакции к рублям, если сделать предположение:   
- что чаще всего используются рубли (11),   
- далее это доллары (1)   
- и евро (14), далее другая валюта. 
- Другими популярными валютами являются, например, белорусские рубли и юани.  
- Кроме долларов и евро, курс у остальных валют колеблется в районе 20 руб.  

По данным ЦБ в среднем курс доллара за рассматриваемый период был 68,3838, а евро за был 72,2106. Можно взять в среднем 70 руб.  

In [62]:
%%time
compress_trx_df['normal_amount'] = np.where(
    compress_trx_df['currency'] == 11, 
    compress_trx_df['amount'],
    np.where(compress_trx_df['currency'].isin([1,14]), 
             compress_trx_df['amount'] * 70,
             compress_trx_df['amount'] * 20
    )
)
compress_trx_df.shape

CPU times: total: 3.77 s
Wall time: 2.25 s


(215076361, 15)

In [78]:
# Загружаем инфляцию
# Например с января 2021 года по января 2024 года рубль инфляция съела 0,3 рубля
inflation_df = pd.read_excel(PATH_DATASET_OUTPUT + 'outside_data/inflation.xlsx').rename(columns={'Месяц': 'month', 'Год': 'year'})

inflation_dict = {}
for y, m, c in inflation_df[['year', 'month', 'cumulative_inflation']].to_dict('split')['data']:
    if y not in inflation_dict:
        inflation_dict[y] = {}
    inflation_dict[y][m] = c
    
inflation_df.shape

(37, 9)

In [90]:
# Нормализация цен по инфляции
def amt_normalize(x):
    # Нормируем к инфляции рублевой
    normal_amt = x.normal_amount * inflation_dict[x.event_time.year][x.event_time.month]
    return normal_amt

compress_trx_df['normal_amount'] = compress_trx_df[['event_time', 'normal_amount']].progress_apply(lambda x: amt_normalize(x), axis=1)
compress_trx_df.shape

100%|██████████| 215076361/215076361 [45:48<00:00, 78238.81it/s] 


(215076361, 15)

In [92]:
%%time
# Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
def series_to_int(col_df:pd.Series):
    """
    Перевод в целочисленные типы
    """
    min_val = col_df.min()
    max_val = col_df.max()
    if min_val >= -128 and max_val <= 127:
        col_df = col_df.astype('int8')
    elif min_val >= -32768 and max_val <= 32767:
        col_df = col_df.astype('int16')
    elif min_val >= -2147483648 and max_val <= 2147483647:
        col_df = col_df.astype('int32')
    else:
        col_df = col_df.astype('int64')
    return col_df

def compression_df(df:pd.DataFrame(), datetime_cols:List[str]=[], category_cols:List[str]=[]):
    """
    Уменьшение размера датафрейма, для таргетов, транзакцй и для фичей
    """
    float64_cols = list(df.select_dtypes(include='float64'))  
    df[float64_cols] = df[float64_cols].astype('float32')
    for col in df.columns:
        if col in category_cols:
            df[col] = df[col].astype('category')
        elif col in datetime_cols:
            if df[col].dtypes == 'object':
                df[col] = pd.to_datetime(df[col])
        # Если колонка содержит числа 
        elif is_integer_dtype(df[col]):
            if df[col].dtypes == 'int8':
                continue
            else:
                df[col] = series_to_int(df[col])
        elif is_float_dtype(df[col]):
            # Возможно ли перевести в число
            if np.array_equal(df[col].fillna(0), df[col].fillna(0).astype(int)):
                df[col] = df[col].fillna(0)
                df[col] = series_to_int(df[col])
    return df

CPU times: total: 15.6 ms
Wall time: 41 ms


In [98]:
compress_trx_df = compression_df(compress_trx_df, 
                            datetime_cols=['report_end' ,'report_next_end', 'event_time'],
                           )
compress_trx_df.shape

(215076361, 15)

In [99]:
compress_trx_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215076361 entries, 0 to 11219876
Data columns (total 15 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[us]
 1   amount         float32       
 2   client_id      category      
 3   event_type     int8          
 4   event_subtype  int8          
 5   currency       int8          
 6   src_type11     int16         
 7   src_type12     int16         
 8   dst_type11     int16         
 9   dst_type12     int16         
 10  src_type21     int32         
 11  src_type22     int8          
 12  src_type31     int16         
 13  src_type32     int8          
 14  normal_amount  float32       
dtypes: category(1), datetime64[us](1), float32(2), int16(5), int32(1), int8(5)
memory usage: 9.5 GB


In [106]:
%%time
compress_trx_df['client_id'] = compress_trx_df['client_id'].astype('object')
compress_trx_df.shape

CPU times: total: 3.2 s
Wall time: 3.26 s


(215076361, 15)

In [108]:
%%time
# Сохраняем в файл оптимизированный файл 
compress_trx_df.to_parquet(PATH_DATASET_OUTPUT + 'compress_norm_trx_df_11_06_2024.parquet')

CPU times: total: 1min 19s
Wall time: 1min 19s
